In [ ]:
import torch

from latentis import PROJECT_ROOT
from pathlib import Path

In [ ]:
exp_dir: Path = PROJECT_ROOT / "results" / "exp3"
exp_dir.exists()

In [ ]:
experiments = list(exp_dir.glob("*"))
len([exp.name for exp in experiments])

In [ ]:
experiments

In [ ]:
import pandas as pd

In [ ]:
from collections import defaultdict


df = defaultdict(list)
for exp_path in experiments:
    if not exp_path.is_file():
        continue
    exp_data = torch.load(exp_path, map_location="cpu", weights_only=False)
    for ablation in exp_data["ablations"]:
        for k, v in ablation.items():
            if k == "keep_units" or k == "ablated_shape":
                continue
            if k == "residual_indices":
                df["n_units"].append(v.numel())
            df[k].append(v if not isinstance(v, torch.Tensor) else v.numpy())
        df["model"].append(exp_data["model_name"])
        df["dataset"].append(exp_data["dataset_name"])
df = pd.DataFrame(df)
df.drop_duplicates(subset=["model", "dataset", "type", "ablation"], inplace=True)
df

In [ ]:
df["type"].unique()

In [ ]:
df["selection_method"] = df["type"].apply(
    lambda x: "_".join(x.split("_")[2:]) if x.startswith("greedy") else "manual"
)
df

In [ ]:
df["n_units"].unique()

In [ ]:
df["selection_method"].unique()

In [ ]:
import wandb

In [ ]:
api = wandb.Api()
runs = api.runs("resi_dual/residual", filters={"config.exp_type": "residual_coarse"})

In [ ]:
from wandb.apis.public.runs import Run

coarse_data = []
for run in runs:
    run: Run

    coarse_data.append(
        {
            "model": run.config["model_name"],
            "dataset": run.config["dataset_name"],
            "type": run.config["exp_type"],
            "score": run.summary.get("test/accuracy", None),
            "selection_method": "optimized",
        }
    )
coarse_data = pd.DataFrame(coarse_data)

coarse_data

In [ ]:
from residual.data.data_registry import dataset_names

filtered_df = df.copy()
filtered_df = filtered_df[filtered_df["model"].isin({"openclip_l", "blip_l_flickr"})]
filtered_df.drop("residual_indices", axis=1, inplace=True)
filtered_df = filtered_df[(filtered_df["ablation"] != "mean")]
types = {
    "greedy_5%_corr_full_out_heads": "U",
    "greedy_5%_corr_task_heads": "U|T",
    "greedy_5%_supervised_heads": "S",
    "random_mean": "R",
    "heads": "H",
    "units": "B",
    "residual_coarse": "O",
    # **{f"greedy_5%_random_{i}_heads": "R" for i in range(10)},
}

random_rows = filtered_df[filtered_df["type"].str.contains("random")]

# Step 1: Filter rows where 'type' contains 'random'
filtered_df = filtered_df[filtered_df["type"].isin(types.keys())]

# Step 2: Group by 'model' and 'dataset' and calculate mean and std for each group
grouped_random = random_rows.groupby(["model", "dataset"])

# Initialize an empty DataFrame to store results
result_df = filtered_df[~filtered_df["type"].str.contains("random")].copy()

# Initialize a list to collect new rows
new_rows = []

# Loop through each group and calculate the mean and std, then append to the result dataframe
for (model, dataset), group in grouped_random:
    # Calculate mean and std for the group
    group_mean = group.mean(numeric_only=True)
    group_std = group.std(numeric_only=True)

    # Prepare new rows for mean and std
    mean_row = pd.Series(group_mean, name=f"random_mean_{model}_{dataset}")
    std_row = pd.Series(group_std, name=f"random_std_{model}_{dataset}")

    # Add 'model' and 'dataset' information
    mean_row["model"] = model
    mean_row["dataset"] = dataset
    mean_row["ablation"] = "zero"
    mean_row["type"] = "random_mean"

    std_row["model"] = model
    std_row["dataset"] = dataset
    std_row["ablation"] = "zero"
    std_row["type"] = "random_std"

    # Append the mean and std rows to the list of new rows
    new_rows.append(mean_row)
    new_rows.append(std_row)

# Step 3: Convert the list of new rows into a DataFrame and concatenate with the result dataframe
new_rows_df = pd.DataFrame(new_rows)
result_df = pd.concat([result_df, new_rows_df], ignore_index=True)
result_df = result_df[result_df["type"] != "random_std"]

result_df

In [ ]:
from residual.nn.model_registry import model_names

result_df = pd.concat(
    [result_df, coarse_data[coarse_data["model"].isin(result_df["model"].unique())]],
    ignore_index=True,
)
result_df.sort_values(by=["model", "dataset", "type"], inplace=True)
result_df["model"] = result_df["model"].apply(lambda x: model_names[x])
result_df["dataset"] = result_df["dataset"].apply(lambda x: dataset_names[x])

result_df["type"] = result_df["type"].apply(types.__getitem__)
result_df["type"] = pd.Categorical(result_df["type"], categories=types.values())
result_df

In [ ]:
table = result_df.pivot(
    index=["dataset"], columns=["model", "type"], values="score"
).fillna(0)
# reorder columns
table = table[
    sorted(table.columns, key=lambda x: (x[0], list(types.values()).index(x[1])))
]
table = table.to_latex(
    multirow=True, column_format="c", multicolumn_format="c", float_format="%.2f"
)
print(table)